IMPORTAZIONE LIBRERIE

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks

CAMBIO PATH PER ANALISI RISPETTO AL LOG DI VOLO CONSIDERATO

In [2]:
path_file = os.path.abspath(os.getcwd())
print(path_file)
os.chdir(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\M2\FAULT_M2_5


VOLO m2 FAULT 10%

In [3]:
rcou_m2_fault5 = pd.read_csv("RCOU.csv")
rcou_m2_fault5 = rcou_m2_fault5.drop(['LineNo','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8'], axis=1)
rcou_m2_fault5 = rcou_m2_fault5[((rcou_m2_fault5['C9'] >= 1350) & (rcou_m2_fault5['C10'] >= 1350) & (rcou_m2_fault5['C11'] >= 1350) & (rcou_m2_fault5['C12'] >= 1350) & (rcou_m2_fault5['C13'] >= 1350) & (rcou_m2_fault5['C14']>= 1350))]
rcou_m2_fault5=rcou_m2_fault5.reset_index(drop=True)
display(rcou_m2_fault5)

,TimeUS,C9,C10,C11,C12,C13,C14
0,70497686,1379,1389,1404,1363,1397,1371
1,70508601,1354,1424,1398,1383,1359,1420
2,70519703,1368,1423,1384,1408,1352,1438
3,70530500,1403,1402,1413,1392,1411,1394
4,70561170,1480,1352,1412,1428,1469,1366
...,...,...,...,...,...,...,...
5038,131358716,1506,1466,1525,1446,1443,1527
5039,131368599,1519,1450,1526,1441,1457,1513
5040,131379386,1541,1415,1523,1437,1480,1484
5041,131396916,1503,1435,1545,1384,1472,1468


In [4]:
min=rcou_m2_fault5['TimeUS'][0]
max=rcou_m2_fault5['TimeUS'][len(rcou_m2_fault5)-1]
print(max)
print(min)

131407710
70497686


In [5]:
xkf1_m2_fault5 = pd.read_csv("XKF1_0.csv")
xkf1_m2_fault5 = xkf1_m2_fault5.drop(['LineNo','C','VN','VE','VD','dPD','PN','PE','PD','GX','GY','GZ','OH'], axis=1)
xkf1_m2_fault5 = xkf1_m2_fault5[((xkf1_m2_fault5['TimeUS'] >= min) & (xkf1_m2_fault5['TimeUS'] <= max	))]
xkf1_m2_fault5 = xkf1_m2_fault5.reset_index(drop=True)
print(xkf1_m2_fault5)

         TimeUS  Roll  Pitch    Yaw
0      70506077  0.86  -0.80  22.99
1      70516653  0.82  -0.78  23.02
2      70527788  0.81  -0.82  23.02
3      70538645  0.87  -0.88  23.03
4      70548617  0.89  -0.88  23.02
...         ...   ...    ...    ...
5534  131356703 -1.94   2.88  22.24
5535  131366708 -1.93   2.76  22.34
5536  131376923 -1.88   2.59  22.41
5537  131394548 -1.80   2.27  22.51
5538  131405296 -1.77   2.09  22.55

[5539 rows x 4 columns]


In [6]:
att_m2_fault5 = pd.read_csv("ATT.csv")
att_m2_fault5 = att_m2_fault5.drop(['LineNo','ErrRP', 'ErrYaw', 'AEKF'], axis=1)
att_m2_fault5 = att_m2_fault5[((att_m2_fault5['TimeUS'] >= min) & (att_m2_fault5['TimeUS'] <= max	))]
att_m2_fault5=att_m2_fault5.reset_index(drop=True)
print(att_m2_fault5)

         TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw
0      70507692     0.00  0.86      0.06  -0.80   22.93  22.99
1      70518797     0.00  0.82      0.06  -0.78   22.93  23.02
2      70529634     0.00  0.81      0.06  -0.82   22.93  23.02
3      70539879     0.01  0.87      0.05  -0.88   22.93  23.03
4      70549895     0.01  0.89      0.05  -0.88   22.93  23.02
...         ...      ...   ...       ...    ...     ...    ...
5537  131358142     1.94 -1.94      0.46   2.88   25.39  22.24
5538  131367962     2.01 -1.93      0.41   2.76   25.39  22.34
5539  131378544     2.06 -1.88      0.37   2.59   25.38  22.41
5540  131396170     2.10 -1.80      0.34   2.27   25.38  22.51
5541  131407152     2.11 -1.77      0.33   2.09   25.38  22.55

[5542 rows x 7 columns]


In [7]:
if len(att_m2_fault5)>len(xkf1_m2_fault5):
    to_add=att_m2_fault5[len(xkf1_m2_fault5):]
    att_m2_fault5=att_m2_fault5[:len(xkf1_m2_fault5)]

if len(xkf1_m2_fault5)>len(att_m2_fault5):
    to_add=xkf1_m2_fault5[len(att_m2_fault5):]

for idx,i in enumerate(att_m2_fault5['Roll']):
    if(xkf1_m2_fault5['Roll'][idx] != i):
        att_m2_fault5['Roll'][idx] = ((att_m2_fault5['Roll'][idx] + xkf1_m2_fault5['Roll'][idx])/2)

for idx,i in enumerate(att_m2_fault5['Pitch']):
    if(xkf1_m2_fault5['Pitch'][idx] != i):
        att_m2_fault5['Pitch'][idx] = ((att_m2_fault5['Pitch'][idx] + xkf1_m2_fault5['Pitch'][idx])/2)

for idx,i in enumerate(att_m2_fault5['Yaw']):
    if(xkf1_m2_fault5['Yaw'][idx] != i):
        att_m2_fault5['Yaw'][idx] = ((att_m2_fault5['Yaw'][idx] + xkf1_m2_fault5['Yaw'][idx])/2)

att_m2_fault5=pd.concat([att_m2_fault5,to_add])

print(att_m2_fault5)

C:\Users\chiar\AppData\Local\Temp\ipykernel_20128\4038585390.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  att_m2_fault5['Roll'][idx] = ((att_m2_fault5['Roll'][idx] + xkf1_m2_fault5['Roll'][idx])/2)
C:\Users\chiar\AppData\Local\Temp\ipykernel_20128\4038585390.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  att_m2_fault5['Pitch'][idx] = ((att_m2_fault5['Pitch'][idx] + xkf1_m2_fault5['Pitch'][idx])/2)
C:\Users\chiar\AppData\Local\Temp\ipykernel_20128\4038585390.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://panda

         TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw     Yaw
0      70507692     0.00  0.86      0.06 -0.800   22.93  22.990
1      70518797     0.00  0.82      0.06 -0.780   22.93  23.020
2      70529634     0.00  0.81      0.06 -0.820   22.93  23.020
3      70539879     0.01  0.87      0.05 -0.880   22.93  23.030
4      70549895     0.01  0.89      0.05 -0.880   22.93  23.020
...         ...      ...   ...       ...    ...     ...     ...
5537  131358142     1.94 -1.87      0.46  2.575   25.39  22.375
5538  131367962     2.01 -1.85      0.41  2.425   25.39  22.445
5539  131378544     2.06 -1.88      0.37  2.590   25.38  22.410
5540  131396170     2.10 -1.80      0.34  2.270   25.38  22.510
5541  131407152     2.11 -1.77      0.33  2.090   25.38  22.550

[5542 rows x 7 columns]


In [8]:
#elettricita' che scorre nel motore considerato, al momento ne consideriamo solo 1
esc_0_m2_fault5 = pd.read_csv("ESC_0.csv")
esc_0_m2_fault5 = esc_0_m2_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_0_m2_fault5 = esc_0_m2_fault5[((esc_0_m2_fault5['TimeUS'] >= min) & (esc_0_m2_fault5['TimeUS'] <= max))]
esc_0_m2_fault5=esc_0_m2_fault5.rename(columns={"RPM": "RPM(ESC_0)", "Curr": "Curr(ESC_0)"})

esc_1_m2_fault5 = pd.read_csv("ESC_1.csv")
esc_1_m2_fault5 = esc_1_m2_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_1_m2_fault5 = esc_1_m2_fault5[((esc_1_m2_fault5['TimeUS'] >= min) & (esc_1_m2_fault5['TimeUS'] <= max))]
esc_1_m2_fault5=esc_1_m2_fault5.rename(columns={"RPM": "RPM(ESC_1)", "Curr": "Curr(ESC_1)"})

esc_2_m2_fault5 = pd.read_csv("ESC_2.csv")
esc_2_m2_fault5 = esc_2_m2_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_2_m2_fault5 = esc_2_m2_fault5[((esc_2_m2_fault5['TimeUS'] >= min) & (esc_2_m2_fault5['TimeUS'] <= max))]
esc_2_m2_fault5=esc_2_m2_fault5.rename(columns={"RPM": "RPM(ESC_2)", "Curr": "Curr(ESC_2)"})

esc_3_m2_fault5 = pd.read_csv("ESC_3.csv")
esc_3_m2_fault5 = esc_3_m2_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_3_m2_fault5 = esc_3_m2_fault5[((esc_3_m2_fault5['TimeUS'] >= min) & (esc_3_m2_fault5['TimeUS'] <= max))]
esc_3_m2_fault5=esc_3_m2_fault5.rename(columns={"RPM": "RPM(ESC_3)", "Curr": "Curr(ESC_3)"})

esc_4_m2_fault5 = pd.read_csv("ESC_4.csv")
esc_4_m2_fault5 = esc_4_m2_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_4_m2_fault5 = esc_4_m2_fault5[((esc_4_m2_fault5['TimeUS'] >= min) & (esc_4_m2_fault5['TimeUS'] <= max))]
esc_4_m2_fault5=esc_4_m2_fault5.rename(columns={"RPM": "RPM(ESC_4)", "Curr": "Curr(ESC_4)"})

esc_5_m2_fault5 = pd.read_csv("ESC_5.csv")
esc_5_m2_fault5 = esc_5_m2_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_5_m2_fault5 = esc_5_m2_fault5[((esc_5_m2_fault5['TimeUS'] >= min) & (esc_5_m2_fault5['TimeUS'] <= max))]
esc_5_m2_fault5=esc_5_m2_fault5.rename(columns={"RPM": "RPM(ESC_5)", "Curr": "Curr(ESC_5)"})

In [9]:
#valori di accelerazione e del giroscopio
imu_0_m2_fault5 = pd.read_csv("IMU_0.csv")
imu_0_m2_fault5 = imu_0_m2_fault5.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_0_m2_fault5 = imu_0_m2_fault5[((imu_0_m2_fault5['TimeUS'] >= min) & (imu_0_m2_fault5['TimeUS'] <= max))]

imu_1_m2_fault5 = pd.read_csv("IMU_1.csv")
imu_1_m2_fault5 = imu_1_m2_fault5.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_1_m2_fault5 = imu_1_m2_fault5[((imu_1_m2_fault5['TimeUS'] >= min) & (imu_1_m2_fault5['TimeUS'] <= max))]

imu_2_m2_fault5 = pd.read_csv("IMU_2.csv")
imu_2_m2_fault5 = imu_2_m2_fault5.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_2_m2_fault5 = imu_2_m2_fault5[((imu_2_m2_fault5['TimeUS'] >= min) & (imu_2_m2_fault5['TimeUS'] <= max))]

imu_m2_fault5 = pd.concat((imu_0_m2_fault5, imu_1_m2_fault5, imu_2_m2_fault5))
imu_m2_fault5=imu_m2_fault5.groupby(imu_m2_fault5.TimeUS, as_index=False).mean()

display(imu_m2_fault5)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ
0,70499925,0.017612,-0.049395,0.006887,-0.126758,0.039685,-9.754690
1,70501976,0.112567,-0.056947,-0.002161,-0.219659,-0.037732,-10.140273
2,70504650,0.057938,-0.035145,0.015399,-0.271173,-0.141826,-10.224187
3,70507590,-0.042021,0.008419,0.032312,-0.280594,-0.294746,-9.969529
4,70510441,-0.043291,-0.072291,0.021327,-0.184740,-0.245387,-9.670237
...,...,...,...,...,...,...,...
21031,131395945,0.058947,-0.343453,0.085216,0.791422,1.257212,-12.164007
21032,131399001,0.059801,-0.340156,0.073685,0.771485,1.439409,-12.753277
21033,131402115,0.108500,-0.310292,0.066445,0.849265,1.520438,-13.021527
21034,131404186,0.137428,-0.350096,0.051299,0.959155,1.500244,-13.369807


In [10]:
#sincronizzazione dei tempi tra att e imu_2
m2_fault5 = pd.merge_ordered(imu_m2_fault5,att_m2_fault5)
m2_fault5=m2_fault5.fillna(method="ffill").fillna(method="bfill")
#sincronizzazione dei tempi tra la tabella ottenuta precedentemente e esc_0 (corrente che passa nel motore)
m2_fault5 = pd.merge_ordered(m2_fault5, esc_0_m2_fault5)
m2_fault5 = pd.merge_ordered(m2_fault5, esc_1_m2_fault5)
m2_fault5 = pd.merge_ordered(m2_fault5, esc_2_m2_fault5)
m2_fault5 = pd.merge_ordered(m2_fault5, esc_3_m2_fault5)
m2_fault5 = pd.merge_ordered(m2_fault5, esc_4_m2_fault5)
m2_fault5 = pd.merge_ordered(m2_fault5, esc_5_m2_fault5)
m2_fault5=m2_fault5.fillna(method="ffill").fillna(method="bfill")
m2_fault5 = pd.merge_ordered(m2_fault5, rcou_m2_fault5)
m2_fault5=m2_fault5.fillna(method="ffill").fillna(method="bfill")

In [11]:
index_to_remove=[]
for ind in att_m2_fault5["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_0_m2_fault5["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_1_m2_fault5["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_2_m2_fault5["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_3_m2_fault5["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_4_m2_fault5["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_5_m2_fault5["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in rcou_m2_fault5["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))

5542
6058
6573
7072
7594
8110
8622


In [12]:
m2_fault5 = m2_fault5[~m2_fault5.TimeUS.isin(index_to_remove)]
m2_fault5["TimeUS"] = m2_fault5["TimeUS"] - m2_fault5.iloc[0]["TimeUS"]
m2_fault5["TimeUS"] = m2_fault5["TimeUS"].astype(int)
m2_fault5["Guasto"] = 1

In [13]:
from datetime import timedelta
m2_fault5=m2_fault5.reset_index(drop=True)
m2_fault5['TimeUS'] = pd.to_datetime(m2_fault5['TimeUS'], unit='us').dt.strftime('%H:%M:%S.%f')
display(m2_fault5)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,...,Curr(ESC_4),RPM(ESC_5),Curr(ESC_5),C9,C10,C11,C12,C13,C14,Guasto
0,00:00:00.000000,0.017612,-0.049395,0.006887,-0.126758,0.039685,-9.754690,0.00,0.86,0.06,...,2.35,2979.0,2.30,1379.0,1389.0,1404.0,1363.0,1397.0,1371.0,1
1,00:00:00.002051,0.112567,-0.056947,-0.002161,-0.219659,-0.037732,-10.140273,0.00,0.86,0.06,...,2.35,2979.0,2.30,1379.0,1389.0,1404.0,1363.0,1397.0,1371.0,1
2,00:00:00.004725,0.057938,-0.035145,0.015399,-0.271173,-0.141826,-10.224187,0.00,0.86,0.06,...,2.35,2979.0,2.30,1379.0,1389.0,1404.0,1363.0,1397.0,1371.0,1
3,00:00:00.007665,-0.042021,0.008419,0.032312,-0.280594,-0.294746,-9.969529,0.00,0.86,0.06,...,2.35,2979.0,2.30,1379.0,1389.0,1404.0,1363.0,1397.0,1371.0,1
4,00:00:00.010516,-0.043291,-0.072291,0.021327,-0.184740,-0.245387,-9.670237,0.00,0.86,0.06,...,2.35,2979.0,2.30,1354.0,1424.0,1398.0,1383.0,1359.0,1420.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21031,00:01:00.896020,0.058947,-0.343453,0.085216,0.791422,1.257212,-12.164007,2.06,-1.88,0.37,...,2.01,5556.0,2.54,1541.0,1415.0,1523.0,1437.0,1480.0,1484.0,1
21032,00:01:00.899076,0.059801,-0.340156,0.073685,0.771485,1.439409,-12.753277,2.10,-1.80,0.34,...,2.01,5556.0,2.54,1503.0,1435.0,1545.0,1384.0,1472.0,1468.0,1
21033,00:01:00.902190,0.108500,-0.310292,0.066445,0.849265,1.520438,-13.021527,2.10,-1.80,0.34,...,2.01,5513.0,2.32,1503.0,1435.0,1545.0,1384.0,1472.0,1468.0,1
21034,00:01:00.904261,0.137428,-0.350096,0.051299,0.959155,1.500244,-13.369807,2.10,-1.80,0.34,...,2.01,5513.0,2.32,1503.0,1435.0,1545.0,1384.0,1472.0,1468.0,1


## Analisi nel tempo e in frequenza

In [14]:
import math
#Funzione che calcola il Root Mean Square (RMS)
def rmsValue(arr, n):
    square = 0
    mean = 0.0
    root = 0.0
    #Calcola il quadrato
    for i in range(0,n):
        square += (arr[i]**2)
    #Calcola la media
    mean = (square / (float)(n))
    #Calcola la radice
    root = math.sqrt(mean)
    return root

## Funzione generica per ottenere la tabellina con le feauture nel tempo e in frequenza

In [15]:
def time_freq_feat(V):
    N=350
    height_threshold=0.0
    tab= pd.DataFrame([], columns=['tempo',f'<>({V})',f'var({V})',f'kurt({V})',f'RMS({V})',f'freq1({V})',f'freq2({V})',f'amp1({V})',f'amp2({V})'])
    for i in range(int(len(m2_fault5)/350)):
        V1=m2_fault5[str(V)].iloc[:(i+1*350)+1]
        mean=V1.mean()
        var=V1.var()
        kurt=V1.kurtosis()
        rms=rmsValue(V1,len(V1))
        yf=fft(V1.values)
        xf=fftfreq(N,1/350)
        peaks_index, properties = find_peaks(yf,height=-200,prominence=0.1)
        
        #for k in range(len(peaks_index)):
        #[print("%4.4f    \t %3.4f" %(freq[peaks_index[i]], properties['peak_heights'][i])) for i in range(len(peaks_index))]
        h_max_peak_1=np.unique(properties['peak_heights'])[-1]
        peak1_dimension = np.where(properties['peak_heights'] == h_max_peak_1)
        if (len(peak1_dimension[0])>1):
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][1]
        else:
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][0]

        h_max_peak_2=np.unique(properties['peak_heights'])[-2] 
        peak2_dimension = np.where(properties['peak_heights'] == h_max_peak_2)

        if (len(peak2_dimension[0])>1):
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][1]
        else:
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][0]

        val={'tempo':i,f'<>({V})':mean,f'var({V})':var,f'kurt({V})':kurt,f'RMS({V})':rms,f'freq1({V})':xf[index1],f'freq2({V})':xf[index2],f'amp1({V})':h_max_peak_1,f'amp2({V})':h_max_peak_2}
        tab=tab.append(val,ignore_index=True)
    return tab

In [16]:
from functools import reduce
GyrX=time_freq_feat('GyrX')
GyrY=time_freq_feat('GyrY')
GyrZ=time_freq_feat('GyrZ')
AccX=time_freq_feat('AccX')	
AccY=time_freq_feat('AccY')
AccZ=time_freq_feat('AccZ')
Roll=time_freq_feat('Roll')
Pitch=time_freq_feat('Pitch')
Yaw=time_freq_feat('Yaw')
rpm_ESC_0=time_freq_feat('RPM(ESC_0)')
curr_ESC_0=time_freq_feat('Curr(ESC_0)')
rpm_ESC_1=time_freq_feat('RPM(ESC_1)')
curr_ESC_1=time_freq_feat('Curr(ESC_1)')
rpm_ESC_2=time_freq_feat('RPM(ESC_2)')
curr_ESC_2=time_freq_feat('Curr(ESC_2)')
rpm_ESC_3=time_freq_feat('RPM(ESC_3)')
curr_ESC_3=time_freq_feat('Curr(ESC_3)')
rpm_ESC_4=time_freq_feat('RPM(ESC_4)')
curr_ESC_4=time_freq_feat('Curr(ESC_4)')
rpm_ESC_5=time_freq_feat('RPM(ESC_5)')
curr_ESC_5=time_freq_feat('Curr(ESC_5)')
merged = [GyrX,GyrY,GyrZ,AccX,AccY,AccZ,Roll,Pitch,Yaw,rpm_ESC_0,curr_ESC_0,rpm_ESC_1,curr_ESC_1,rpm_ESC_2,curr_ESC_2,rpm_ESC_3,curr_ESC_3,rpm_ESC_4,curr_ESC_4,rpm_ESC_5,curr_ESC_5]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['tempo'],how='outer'), merged)
df_merged["Guasto"] = 1
display(df_merged)

c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_20128\2524513146.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_20128\2524513146.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards 

,tempo,<>(GyrX),var(GyrX),kurt(GyrX),RMS(GyrX),freq1(GyrX),freq2(GyrX),amp1(GyrX),amp2(GyrX),<>(GyrY),...,amp2(RPM(ESC_5)),<>(Curr(ESC_5)),var(Curr(ESC_5)),kurt(Curr(ESC_5)),RMS(Curr(ESC_5)),freq1(Curr(ESC_5)),freq2(Curr(ESC_5)),amp1(Curr(ESC_5)),amp2(Curr(ESC_5)),Guasto
0,0.0,-0.007274,0.005704,0.065217,0.075767,66.0,88.0,4.179859,1.708813,0.047110,...,14103.680636,3.113504,0.314603,-0.981229,3.163482,124.0,121.0,40.413568,6.483713,1
1,1.0,-0.007474,0.005702,0.057401,0.075773,60.0,79.0,3.534962,1.716919,0.046805,...,14567.586147,3.113778,0.313734,-0.975669,3.163615,122.0,119.0,40.476864,6.277084,1
2,2.0,-0.007871,0.005741,0.033471,0.076072,65.0,64.0,3.940210,1.752200,0.046627,...,14985.327916,3.114051,0.312868,-0.970107,3.163747,128.0,125.0,40.530103,6.069243,1
3,3.0,-0.008282,0.005785,0.010565,0.076401,66.0,67.0,3.208691,2.804103,0.046534,...,15353.376178,3.114322,0.312008,-0.964542,3.163879,128.0,125.0,40.573298,5.863191,1
4,4.0,-0.008509,0.005787,-0.000253,0.076439,66.0,67.0,4.336626,2.584081,0.046293,...,15668.647294,3.114592,0.311153,-0.958974,3.164010,128.0,125.0,40.606468,5.661653,1
5,5.0,-0.008657,0.005778,-0.001615,0.076400,65.0,86.0,3.832512,1.894932,0.045927,...,15928.542786,3.114860,0.310302,-0.953405,3.164140,128.0,125.0,40.629629,5.467071,1
6,6.0,-0.009027,0.005811,-0.024206,0.076657,59.0,77.0,3.649880,1.908404,0.045802,...,16036.701384,3.115126,0.309455,-0.947832,3.164269,128.0,125.0,40.642803,5.281597,1
7,7.0,-0.009511,0.005879,-0.034878,0.077154,65.0,64.0,3.758501,2.273350,0.045771,...,15795.417569,3.115391,0.308614,-0.942258,3.164398,130.0,123.0,40.646014,5.432039,1
8,8.0,-0.009827,0.005898,-0.054470,0.077319,67.0,68.0,3.921583,2.558451,0.045586,...,15543.697027,3.115655,0.307777,-0.936681,3.164526,130.0,124.0,40.639286,5.565863,1
9,9.0,-0.009989,0.005891,-0.057386,0.077295,71.0,72.0,4.440221,2.646954,0.045322,...,15281.633533,3.115917,0.306944,-0.931101,3.164653,130.0,124.0,40.622648,5.545823,1


In [17]:
path_file = path_file.replace(r"M2\FAULT_M2_5", "")
os.chdir(path_file)
df_merged.to_csv('m2_fault5.csv', index=False)
print(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\
